- True implementation
  - LightGBM (scikit learn)
  - Loop: attack & train
    - annotate full-attacks dataset with costs
    - generate attacked datasets
    - continue training from previous forest with new data
    
    
validation split is a bit strange ...

In [1]:
import os
import numpy as np
import pandas as pd
import lightgbm

## Prepare Data

In [2]:
def infer_categorical_features(dataset):
    categorical_features = set([])
    for column in dataset.columns:
        if dataset[column].dtype == 'object':
            categorical_features.add(column)
    return categorical_features
            
def label_encode(dataset, categorical_features):
    dataset_le = dataset.copy()
    for column in dataset_le.columns:
        if column in categorical_features:
            dataset_le[column] = dataset_le[column].astype('category')
            dataset_le[column] = dataset_le[column].cat.codes
    return dataset_le

def load_atk_train_valid_test( atk_train_file, atk_valid_file, atk_test_file, force=False):
    
    
    if  ( force or 
          not os.path.exists(atk_train_file+".cat.bz2") or
          not os.path.exists(atk_valid_file+".cat.bz2") or
          not os.path.exists(atk_test_file+".cat.bz2") ):
    
        print ("Pre-processing original files...")

        print ("Loading:", atk_train_file)
        print ("Loading:", atk_valid_file)
        print ("Loading:", atk_test_file)

        train = pd.read_csv(atk_train_file)
        valid = pd.read_csv(atk_valid_file)
        test  = pd.read_csv(atk_test_file)
        
        print ("Train/Valid/Test sizes:", train.shape, valid.shape, test.shape)
        print ("Train/Valid/Test split: {:.2f} {:.2f} {:.2f}"
                   .format( train.shape[0]/(train.shape[0]+valid.shape[0]+test.shape[0]),
                            valid.shape[0]/(train.shape[0]+valid.shape[0]+test.shape[0]),
                            test.shape[0] /(train.shape[0]+valid.shape[0]+test.shape[0]) ) )

        # concat to process correctly label encoding
        full = pd.concat( [train, valid, test] )

        fx = infer_categorical_features(full)
        print("List of categorical features: [{}]".format(", ".join([cf for cf in fx])))

        full = label_encode(full, fx)

        # split-back into train valid test
        train_cat = full.iloc[0:train.shape[0],:]
        valid_cat = full.iloc[train.shape[0]:train.shape[0]+valid.shape[0],:]
        test_cat  = full.iloc[train.shape[0]+valid.shape[0]:,:]    

        # save to file
        print ("Saving processed files *.cat.bz2")
        train_cat.to_csv(atk_train_file+".cat.bz2", compression="bz2", index=False)
        valid_cat.to_csv(atk_valid_file+".cat.bz2", compression="bz2", index=False)
        test_cat.to_csv (atk_test_file+".cat.bz2",  compression="bz2", index=False)
        
    else:
        print ("Loading pre-processed files...")

        train_cat = pd.read_csv(atk_train_file+".cat.bz2")
        valid_cat = pd.read_csv(atk_valid_file+".cat.bz2")
        test_cat  = pd.read_csv(atk_test_file+".cat.bz2")
    
    # return data
    return train_cat, valid_cat, test_cat


## Adversarial Boosting

In [3]:
def AdvBoosting_gen_data(model, data, groups):
    ''' 
    model  : is the LightGBM Model
    data   : data matrix with all valid attacks (last column is label)
    groups : grouping of same attacked instance 
    returns the new data matrix and new groups
    
    WARNING: currently works only for binary classification
    '''
    # score the datataset
    labels = data[:,-1]
    
    predictions = model.predict(data[:,:-1]) # exclude labels
    # binarize
    predictions = (predictions>0).astype(np.float)
    predictions = 2*predictions - 1
    
    # check mispredictions
    matchings = labels * predictions
    
    # select original data + attacked instances
    new_selected = [] # id of selected instances
    new_groups   = []
    
    offset = 0
    for g in groups:
        if g==1:
            # there are not attacks, just add original
            new_selected += [offset]
            new_groups   += [1]
        else:
            # get a slice of the matching scores
            g_matchings = matchings[offset:offset+g]

            # most misclassified (smallest margin)
            # skip original
            adv_instance = np.argmin(g_matchings[1:])+1

            # add original and adversarial
            new_selected += [offset, adv_instance]
            new_groups   += [2]
        
        offset += g
    
    new_dataset = data[new_selected,:]
    
    return new_dataset, new_groups

In [4]:
# Our custom metrics
def binary_log_loss(pred, true_label):

    return np.log(1.0 + np.exp(-pred * true_label))

# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss(preds, train_data):
    
    labels = train_data.get_label()
    losses = np.log(1.0 + np.exp(-preds*labels))
    avg_loss = np.mean(losses)
    
    return 'avg_binary_log_loss', avg_loss, False


def optimize_log_loss(preds, train_data):
    labels = train_data.get_label()
    exp_pl = np.exp(preds * labels)
    # http://www.wolframalpha.com/input/?i=differentiate+log(1+%2B+exp(-kx)+)
    grads = -labels / (1.0 +  exp_pl)  
    # http://www.wolframalpha.com/input/?i=d%5E2%2Fdx%5E2+log(1+%2B+exp(-kx)+)
    hess = labels**2 * exp_pl / (1.0 + exp_pl)**2 

    # this is to optimize average logloss
    norm = 1.0/len(preds)
    grads *= norm
    hess *= norm
    
    return grads, hess


# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss_uma(preds, train_data):
    labels = train_data.get_label()
    attack_lens = train_data.get_group()
    
    offset = 0
    max_logloss = []
    avg_max_logloss = 0.0
    
    if attack_lens is not None:
    
        for atk in attack_lens:
            losses = [binary_log_loss(h,t) for h,t in zip(preds[offset:offset+atk], labels[offset:offset+atk])]
            max_logloss.append(max(losses))

            offset += atk
        
        avg_max_logloss = np.mean(max_logloss)  

    return 'avg_binary_log_loss_under_max_attack', avg_max_logloss, False

def avg_non_interferent_log_loss(preds, train_data, alpha=1.0):
    
    # binary logloss under maximal attack
    _, loss_uma, _    = avg_log_loss_uma(preds, train_data)
    
    # binary logloss (plain)
    _, loss_plain, _  = avg_log_loss(preds, train_data)
    
    # combine the above two losses together
    weighted_loss = alpha*loss_uma + (1.0-alpha)*loss_plain

    return 'avg_non_interferent_log_loss [alpha={}]'.format(alpha), weighted_loss, False

def optimize_log_loss_uma(preds, train_data):
    labels = train_data.get_label()
    attack_lens = train_data.get_group()
    
    grads = np.zeros_like(labels, dtype=np.float64)
    hess = np.zeros_like(grads)
    
    if attack_lens is not None:

        norm = 1.0 / float(len(attack_lens))

        offset = 0
        for atk in attack_lens:
            exp_pl = np.exp(- preds[offset:offset+atk] * labels[offset:offset+atk])

            inv_sum = 1.0 / np.sum(1.0 + exp_pl)

            x_grad = inv_sum * exp_pl

            grads[offset:offset+atk] = norm * x_grad * (- labels[offset:offset+atk])
            hess[offset:offset+atk]  = norm * x_grad * (1.0 - x_grad)

            offset += atk    
    
    return grads, hess

def optimize_non_interferent_log_loss(preds, train_data, alpha=1.0):
    # binary logloss under maximal attack
    grads_uma, hess_uma = optimize_log_loss_uma(preds, train_data)
    
    # binary logloss (plain)
    grads_plain, hess_plain = optimize_log_loss(preds, train_data)
    
    # combine the above two losses together
    grads = alpha*grads_uma + (1.0-alpha)*grads_plain
    hess  = alpha*hess_uma  + (1.0-alpha)*hess_plain
    
    return grads, hess



def AdvBoosting_extend_model(data, input_model=None, num_trees=1, params=None):
    ''' 
    model  : is the LightGBM Model
    data   : data matrix with all valid attacks (last column is label)
    returns the new model (is model modified inplace?)
    '''
    
    if params is None:
        params = {
            'learning_rate': 0.1,
            'num_leaves': 16,
            'min_data_in_leaf': 20, #[1, 20]
            'verbose': 0
        }  

    lgbm_train = lightgbm.Dataset(data=data[:,:-1], 
                                  label=data[:,-1])
    
    lgbm_info = {}
    lgbm_model = lightgbm.train(params, lgbm_train, 
                                num_boost_round = num_trees, 
                                init_model = input_model,
                                fobj = optimize_log_loss, 
                                feval = avg_log_loss,
                                evals_result = lgbm_info,
                                valid_sets   = [lgbm_train], 
                                valid_names  = ['adv-train'],
                                verbose_eval=10)

    return lgbm_model, lgbm_info

In [5]:
def AdvBoosting( atk_train, trees, 
                 output_model_file,
                 partial_save=10, 
                 adv_rounds=1):
    ''' 
    atk_data: full dataset including all valid attacks
    atk_groups: lenght of each attack set
    trees: total number of trees to be produced
    adv_rounds: adversarial instance injecting frequency
    '''
    # get groups and remove instance ids
    atk_groups = atk_train['instance_id'].value_counts().sort_index().values
    atk_data   = atk_train.iloc[:,1:].values
    
    # train first trees
    original_ids = np.cumsum(atk_groups[:-1])
    original_ids = np.insert(original_ids, 0, 0)
    
    model, model_info = AdvBoosting_extend_model( atk_data[original_ids, :], 
                                                  input_model=None, 
                                                  num_trees=adv_rounds, 
                                                  params=None )
    
    # train remaining trees
    for t in range(2, trees+1, adv_rounds):
        # attack dataset
        adv_data, adv_offsets = AdvBoosting_gen_data( model, atk_data, atk_groups )
        
        # train additional trees
        model, model_info = AdvBoosting_extend_model( adv_data, 
                                                      input_model=model, 
                                                      num_trees=adv_rounds, 
                                                      params=None)
        # save partial model
        if t%partial_save==0 and t!=trees:
            partial_filename = "{}.T{:03d}.lgbm".format(output_model_file, t)
            if os.path.exists(partial_filename):
                os.remove(partial_filename)
            model.save_model( filename=partial_filename )
            
    if os.path.exists(output_model_file):
        os.remove(output_model_file)  
    model.save_model(filename=output_model_file)

# Start the Training

In [6]:
DATASETS_PATH = "../data/census"
MODELS_PATH = "../out/models"
ATTACKER = "strong" # weak


TRAINING_SET="train_ori.csv.bz2" # original training set
TRAINING_SET_ATT="train_"+ATTACKER+"_att.csv.bz2" # perturbed training set
VALIDATION_SET="valid_ori.csv.bz2" # original validation set
VALIDATION_SET_ATT="valid_"+ATTACKER+"_att.csv.bz2" # perturbed validation set
TEST_SET="test_ori.csv.bz2" # original test set
TEST_SET_ATT="test_"+ATTACKER+"_att.csv.bz2" # perturbed test set

In [7]:
train, valid, test = load_atk_train_valid_test(  
                                os.path.join(DATASETS_PATH, TRAINING_SET_ATT),
                                os.path.join(DATASETS_PATH, VALIDATION_SET_ATT),
                                os.path.join(DATASETS_PATH, TEST_SET_ATT))

Loading pre-processed files...


In [8]:
adv_model = "../out/models/adv_boosting.lgbm"

AdvBoosting(train, trees=100, output_model_file=adv_model, adv_rounds=100, partial_save=100)

[10]	adv-train's avg_binary_log_loss: 0.414062
[20]	adv-train's avg_binary_log_loss: 0.343473
[30]	adv-train's avg_binary_log_loss: 0.317184
[40]	adv-train's avg_binary_log_loss: 0.304094
[50]	adv-train's avg_binary_log_loss: 0.297048
[60]	adv-train's avg_binary_log_loss: 0.292382
[70]	adv-train's avg_binary_log_loss: 0.289018
[80]	adv-train's avg_binary_log_loss: 0.286211
[90]	adv-train's avg_binary_log_loss: 0.283709
[100]	adv-train's avg_binary_log_loss: 0.281838
[110]	adv-train's avg_binary_log_loss: 0.143443
[120]	adv-train's avg_binary_log_loss: 0.140765
[130]	adv-train's avg_binary_log_loss: 0.139608
[140]	adv-train's avg_binary_log_loss: 0.139014
[150]	adv-train's avg_binary_log_loss: 0.138433
[160]	adv-train's avg_binary_log_loss: 0.137954
[170]	adv-train's avg_binary_log_loss: 0.137515
[180]	adv-train's avg_binary_log_loss: 0.137123
[190]	adv-train's avg_binary_log_loss: 0.136724
[200]	adv-train's avg_binary_log_loss: 0.136319


In [9]:
!ls -lht ../out/models

total 684K
-rw-rw-r-- 1 lucchese lucchese 283K Feb 16 09:52 adv_boosting.lgbm
-rw-rw-r-- 1 lucchese lucchese  13K Feb 16 09:51 adv_boosting.lgbm.T009.lgbm
-rw-rw-r-- 1 lucchese lucchese  12K Feb 16 09:51 adv_boosting.lgbm.T008.lgbm
-rw-rw-r-- 1 lucchese lucchese  11K Feb 16 09:51 adv_boosting.lgbm.T007.lgbm
-rw-rw-r-- 1 lucchese lucchese 8.8K Feb 16 09:51 adv_boosting.lgbm.T006.lgbm
-rw-rw-r-- 1 lucchese lucchese 7.4K Feb 16 09:51 adv_boosting.lgbm.T005.lgbm
-rw-rw-r-- 1 lucchese lucchese 6.0K Feb 16 09:51 adv_boosting.lgbm.T004.lgbm
-rw-rw-r-- 1 lucchese lucchese 4.6K Feb 16 09:51 adv_boosting.lgbm.T003.lgbm
-rw-rw-r-- 1 lucchese lucchese 3.3K Feb 16 09:51 adv_boosting.lgbm.T002.lgbm
-rw-rw-r-- 1 lucchese lucchese  85K Feb 16 09:50 adv_boosting.lgbm.T060.lgbm
-rw-rw-r-- 1 lucchese lucchese  57K Feb 16 09:50 adv_boosting.lgbm.T040.lgbm
-rw-rw-r-- 1 lucchese lucchese  29K Feb 16 09:50 adv_boosting.lgbm.T020.lgbm
-rw-rw-r-- 1 lucchese lucchese 139K Feb 12 09:41 std_strong_200.pkl


In [10]:
# debug running time
#%load_ext line_profiler
# %lprun -f AdvBoosting  AdvBoosting(train, trees=3)
# %lprun -f AdvBoosting_gen_data  AdvBoosting(train, trees=3)

In [11]:
#!rm ../out/models/*.lgbm